In [21]:



import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device





device(type='cuda')

In [22]:

name = "google/flan-t5-large"

path = f'./data/{name}/base_model'

tokenizer = AutoTokenizer.from_pretrained(name, cache_dir = path)
model = AutoModelForSeq2SeqLM.from_pretrained(name ,cache_dir = path,  device_map="auto" )


In [23]:
input_text = "la capitale du maroc est Rabat Q: donne moi une ville du maroc"

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_new_tokens=200)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Rabat


In [76]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
name = "google/flan-t5-large"
path = f'./data/{name}/base_model'


hf = HuggingFacePipeline.from_model_id(
    model_id=name,
    task="text2text-generation",
    pipeline_kwargs={"max_new_tokens": 10},
model_kwargs={"cache_dir" : path}
    
)

In [70]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = name
tokenizer = AutoTokenizer.from_pretrained(name, cache_dir = path)
model = AutoModelForSeq2SeqLM.from_pretrained(name ,cache_dir = path,  device_map="auto" )


pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
hf = HuggingFacePipeline(pipeline=pipe)

In [73]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate



question = "Who won the FIFA World Cup in the year 1994? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

In [74]:
llm_chain = LLMChain(prompt=prompt, llm=hf)
print(llm_chain.run(question))

ValueError: The following `model_kwargs` are not used by the model: ['return_full_text'] (note: typos in the generate arguments will also show up in this list)

In [75]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

ValueError: The following `model_kwargs` are not used by the model: ['return_full_text'] (note: typos in the generate arguments will also show up in this list)